In [56]:
print(ord('a'))
print(ord('z'))

97
122


In [71]:
#All 3 letter triplets
trips = set()
illegal = {'i', 'o', 'l'}

for i in range(97, 122):
    trips.add(f"{chr(i)}{chr(i+1)}{chr(i+2)}")

    
def isStraight(s:str) -> bool:
    straight = False
    for ind, c in enumerate(s[0:-2]):
        if f"{c.lower()}{s[ind+1].lower()}{s[ind+2].lower()}" in trips:
            straight = True
    return straight

def isNotIllegal(s:str) -> bool:
    not_illegal = True
    for c in s:
        if c in illegal:
            not_illegal = False
    return not_illegal

In [67]:
print(isStraight('ABC'))
print(isStraight('Abc'))
print(isStraight('abC'))
print(isStraight('abc'))
print(isStraight('Acb'))
print(isStraight('acb'))
print(isStraight('aabc'))
print(isStraight('adkghijdg'))

True
True
True
True
False
False
True
True


In [72]:
print(isNotIllegal('aaksdjlahdfblajshbfl'))
print(isNotIllegal('asnxvxgdfdf'))
print(isNotIllegal('aaaaaaaaaaaaaaaaaaaaaaaaaaaaaaaaaaaaaal'))





False
True
False


In [74]:
def doubles(s:str) -> bool:
    doubles = {}
    for c in set(list(s)):
        doubles.add(f'{c + c}')

{'a', 'b', 'c', 'd', 'e'}